In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cancer-data/Cancer_Data.csv


# Import the libraries

In [8]:
import pandas as pd #managing the data
import numpy as np #mathematical operations
import matplotlib.pyplot as plt #vizualization
import seaborn as sns #visualization

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# Evaluation metrics related methods
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_recall_fscore_support, precision_score, recall_score

# Load the data

In [9]:
data = pd.read_csv("/kaggle/input/cancer-data/Cancer_Data.csv")
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


# About the data

"Cancer Data" dataset from Kaggle will be used. Dataset contains informations about cancer patients and their characteristics. 

The dataset includes the following features:

- ID: Patient ID;
- diagnosis: Cancer types (B - benign cancer/M - malignant cancer);
- radius_mean: Visual Characteristics of cancer;
- texture_mean: Visual Characteristics of cancer;
- perimeter_mean: Visual Characteristics of cancer;
- area_mean: Visual Characteristics of cancer;
- smoothness_mean: Visual Characteristics of cancer;
- compactness_mean: Visual Characteristics of cancer;
- concavity_mean: Visual Characteristics of cancer;
- concave_points_mean: Visual Characteristics of cancer;
- symmetry_mean;
- fractial_dimension_mean;
- radius_se;
- texture_se;
- perimeter_se;
- area_se;
- smoothness_se;
- compactness_se;
- concavity_se;
- concave points_se;
- symmetry_se;
- fractal_dimension_se;
- radius_worst;
- texture_worst;
- perimeter_worst;
- area_worst;
- smoothness_worst;
- compactness_worst;
- concavity_worst;
- concave points_worst;
- symmetry_worst;
- fractal_dimension_worst;

# Objectives

The main task is to build a predictive model using the Cancer dataset from Kaggle to accurately classify whether a tumor is malignant or benign. We will train several ML models(KNN and SVM), and compare their results. 


K-Nearest Neighbors (KNN) classifies new cancer example based on the majority class of its k closest neighbors, while Support Vector Machine (SVM) finds the optimal hyperplane that best separates the data into different classes. Both models do well with binary problems.


# Data exploration and preprocessing

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

The dataset contains 569 entries and 33 columns. We have only one object dtype, and we will use label encoding for that in the Feature Engineering part. Null values are present only in the "Unnamed: 32" column, so we will delete that column, as it is not relevant to our task and may cause us a problem in the future.


In [ ]:
#droping the column "Unnamed: 32 "
data = data.drop('Unnamed: 32', axis=1)

In [ ]:
data.columns

In [ ]:
#checking for duplicates 
data.duplicated().sum()

Excellent, no duplicates were found in the dataset.


In [ ]:
data.diagnosis.value_counts(normalize=True)

In [ ]:
data.diagnosis.value_counts().plot.bar(color=['green', 'red'])

We notice that benign tumors are more common, even 0.6%, which is great news, because they are less dangerous, unlike malignant ones.


## Feature Engineering

Since the ML models we will use require numeric variables, we will use the label encoder to convert them from categorical to numeric values. Label encoder assigns a unique integer to each category, allowing the model to use these encoded values for training and prediction.


### Label Encoding

In [ ]:
# Get list of categorical variables
s = (data.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
labeled_data = data.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
labeled_data[object_cols] = ordinal_encoder.fit_transform(data[object_cols])

In [ ]:
labeled_data.head()

### Correlations checking

In order to get the most accurate model and to not overload it with not important informations, we want to keep only the most relevant features. We will filter the features and keep those that have a correlation greater than 0.5 with the target. Target in our case is column "diagnosis". For the same reason, for the most relevant features, we will check the presence of outliers and remove them.



In [ ]:
#list of top features that have high correlation between features and target
features = labeled_data.corr()['diagnosis'].sort_values()
features

In [ ]:
high_corr_features = features[features >= 0.5].index
high_corr_features

In [ ]:
# Visualizing high_corr_features to see if we have outliers

# Ensure all selected columns are numeric
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns

# Filter out non-numeric columns from high_corr_features
high_corr_features = [col for col in high_corr_features if col in numeric_cols]

for column in high_corr_features:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=data[column])
    plt.title(f'Box Plot for {column}')
    plt.show()

### Dealing with outliers - IQR

In [ ]:
#perimeter_se column

#first(q1) quartile, third(q3) quartile and interquartile range(iqr)
q1 = data['perimeter_se'].quantile(0.25)
q3 = data['perimeter_se'].quantile(0.75)
iqr = q3-q1
q1, q3, iqr

#upper and lower bounds for outliers
upper_limit = q3 + (1.5 * iqr)
lower_limit = q1 - (1.5 * iqr)
lower_limit, upper_limit

#data without outliers
labeled_data = labeled_data.loc[(labeled_data['perimeter_se'] < upper_limit) & (labeled_data['perimeter_se'] > lower_limit)]

In [ ]:
#area_se column

#first(q1) quartile, third(q3) quartile and interquartile range(iqr)
q1 = data['area_se'].quantile(0.25)
q3 = data['area_se'].quantile(0.75)
iqr = q3-q1
q1, q3, iqr

#upper and lower bounds for outliers
upper_limit = q3 + (1.5 * iqr)
lower_limit = q1 - (1.5 * iqr)
lower_limit, upper_limit

#data without outliers
labeled_data = labeled_data.loc[(labeled_data['area_se'] < upper_limit) & (labeled_data['area_se'] > lower_limit)]

In [ ]:
#radius_se column

#first(q1) quartile, third(q3) quartile and interquartile range(iqr)
q1 = data['radius_se'].quantile(0.25)
q3 = data['radius_se'].quantile(0.75)
iqr = q3-q1
q1, q3, iqr

#upper and lower bounds for outliers
upper_limit = q3 + (1.5 * iqr)
lower_limit = q1 - (1.5 * iqr)
lower_limit, upper_limit

#data without outliers
labeled_data = labeled_data.loc[(labeled_data['radius_se'] < upper_limit) & (labeled_data['radius_se'] > lower_limit)]

I removed outliers from those features(perimeter_se, area_se, radius_se) where I concluded that there is the greatest deviation based on the box plot.

### Defining raw input and output

In [ ]:
# Define X_raw as a copy of the data without the 'diagnosis' column
X_raw = labeled_data[['area_se', 'perimeter_se', 'radius_se', 'compactness_worst',
       'compactness_mean', 'concavity_worst', 'concavity_mean', 'area_mean',
       'radius_mean', 'area_worst', 'perimeter_mean', 'radius_worst',
       'concave points_mean', 'perimeter_worst', 'concave points_worst']]
                  
# Define y_raw as the 'diagnosis' column                  
y = labeled_data['diagnosis']

### Scaling Features

Scaling features ensures that all features contribute equally to the model and that the model can effectively learn from and generalize to new data. This preprocessing step is crucial for achieving optimal performance in classification tasks. Min-max scaler transforms features by scaling them to a specified range (0, 1).
- Scaling is crucial for KNN model because it relies on distance metrics between feature vectors, and unscaled features can lead to biased influence from features with larger scales. 
- Scaling is important for SVM (Support Vector Machines) because it optimizes a decision boundary by maximizing the margin between classes, and unscaled features can cause the model to prioritize features with larger scales.







In [ ]:
# MinMaxScaler object
scaler = MinMaxScaler()

# Scaling the raw input features
X = scaler.fit_transform(X_raw)

#schecking what we done
print(f"The range of feature inputs are within {X_scaled.min()} to {X_scaled.max()}")

# Models

## Spliting data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = 123)

## KNN

### Model training

In [ ]:
# Defining a KNN model with `n_neighbors=2`
knn_model = KNeighborsClassifier(n_neighbors=2)

knn_model.fit(X_train, y_train.values.ravel())
preds = knn_model.predict(X_test)

### Model evaluaton

- Accuracy: The proportion of correctly classified examples;
- Precision: The proportion of true positive predictions among all positive predictions;
- Recall: The proportion of true positive predictions among all actual positives;
- F1 Score: Balance between precision and recall.


In [ ]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp, average='binary')
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

evaluate_metrics(y_test, preds)

### Tuning the number of K

In [ ]:
# K number
max_k = 50
# empty list to store f1score for each k
f1_scores = []

In [ ]:
for k in range(1, max_k + 1):

    knn = KNeighborsClassifier(n_neighbors=k)
    knn = knn.fit(X_train, y_train.values.ravel())
    preds = knn.predict(X_test)
    
    # Evaluate the model with f1score
    f1 = f1_score(preds, y_test)
    f1_scores.append((k, round(f1_score(y_test, preds), 4)))
    
# Convert the f1score list to a dataframe
f1_results = pd.DataFrame(f1_scores, columns=['K', 'F1 Score'])
f1_results.set_index('K')

Given that K=4 has the highest F1 Score, we will train the model with that value.


In [ ]:
model = KNeighborsClassifier(n_neighbors=4)
model.fit(X_train, y_train.values.ravel())
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)

## SVM

### Model training

In [ ]:
model = SVC(C=2, kernel='rbf')
model.fit(X_train, y_train.values.ravel())
preds = model.predict(X_test)

### Model evaluation

In [ ]:
def evaluate_metrics(yt, yp):
    results_pos = {}
    results_pos['accuracy'] = accuracy_score(yt, yp)
    precision, recall, f_beta, _ = precision_recall_fscore_support(yt, yp, average='binary')
    results_pos['recall'] = recall
    results_pos['precision'] = precision
    results_pos['f1score'] = f_beta
    return results_pos

evaluate_metrics(y_test, preds)

### Tuning C and kernel

In [ ]:
params_grid = {
    'C': [1, 10, 100, 150],
    'kernel': ['poly', 'rbf', 'sigmoid']
}

In [ ]:
model = SVC()

In [ ]:
# Define a GridSearchCV to search the best parameters
grid_search = GridSearchCV(estimator = model, 
                           param_grid = params_grid, 
                           scoring='f1',
                           cv = 5, verbose = 1)

# Search the best parameters with training data
grid_search.fit(X_train, y_train.values.ravel())
best_params = grid_search.best_params_

In [ ]:
best_params

In [ ]:
model = SVC(C=10, kernel='rbf')
model.fit(X_train, y_train.values.ravel())
preds = model.predict(X_test)
evaluate_metrics(y_test, preds)

# Key Findings

Among the models evaluated, the SVM classifier emerged as the best model with the highest accuracy, precision, recall, and F1 score. Here are the detailed metrics for the model:

- Accuracy: 96%
- Precision: 90%
- Recall: 96%
- F1 Score: 93%

### Future Work

- Additional Data: Adding data can help in training a more generalized model, reducing the risk of overfitting;
- Alternative Models: Exploring ensemble methods or deep learning approaches can potentially improve performances;
- Feature Engineering: Investigating additional features or transformations could provide better model accuracy.